Generate 112 trials (for now 1 only subject)

trials with shock after n seconds change physio signals
trials w\out shock no change in physio

In [1]:
# Load NeuroKit and other useful packages
import neurokit2 as nk
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random

In [2]:
TRIAL_LEN = 6
SHOCK_TIME = 4
AFTER_SHOCK = TRIAL_LEN-SHOCK_TIME
BPM_LOW = list(range(50,70))
BPM_HIGH = list(range(80,110))
DRIFT_RANGE= list(map(lambda x: x/1000.0, range(1, 10, 1)))

SEED = 1234
SAMPLING_RATE_HR = 500
SAMPLING_RATE_EDA = 500
THRESHOLD_BINARY = 2
assert SHOCK_TIME <= TRIAL_LEN

In [3]:
def generate_fake_eda_hr_shock(noise_hr=0.01,noise_eda=0.05,seed=None):
    random.seed(seed)
    ecg_low = nk.ecg_simulate(duration=SHOCK_TIME, noise=noise_hr, heart_rate=random.choice(BPM_LOW),sampling_rate=SAMPLING_RATE_HR)
    random.seed(seed)
    ecg_high = nk.ecg_simulate(duration=TRIAL_LEN-SHOCK_TIME, noise=noise_hr, heart_rate=random.choice(BPM_HIGH),sampling_rate=SAMPLING_RATE_HR)
    ecg=np.concatenate([ecg_low,ecg_high])

    while True:
        random.seed(seed)
        eda_test = nk.eda_simulate(duration=20, scr_number=2, noise=noise_eda,drift=random.choice(DRIFT_RANGE))
        signals, info = nk.eda_process(eda_test, sampling_rate=SAMPLING_RATE_EDA)

        if (info['SCR_Recovery'][0] is not None) and (info['SCR_Onsets'][1] is not None) and (not pd.isna(info['SCR_Recovery'][0])) and (not pd.isna(info['SCR_Onsets'][1])) :
            peak = int(info['SCR_Recovery'][0])
            peak_ = int(info['SCR_Onsets'][1])
            if peak_- peak > SAMPLING_RATE_EDA*TRIAL_LEN:
                break

    eda_low = eda_test[peak:peak_]
    eda_low = eda_low[-SHOCK_TIME*SAMPLING_RATE_EDA:]
    eda_high = eda_test[peak_:peak_+AFTER_SHOCK*SAMPLING_RATE_EDA]
    eda = np.concatenate([eda_low,eda_high])

    return ecg,eda

In [4]:
def generate_fake_eda_hr_noshock(noise_hr=0.01,noise_eda=0.05,seed=None):
    random.seed(seed)
    ecg = nk.ecg_simulate(duration=TRIAL_LEN, noise=noise_hr, heart_rate=random.choice(BPM_LOW),sampling_rate=SAMPLING_RATE_HR)

    while True:
        random.seed(seed)
        eda_test = nk.eda_simulate(duration=20, scr_number=2, noise=noise_eda,drift=random.choice(DRIFT_RANGE))
        signals, info = nk.eda_process(eda_test, sampling_rate=SAMPLING_RATE_EDA)

        if (info['SCR_Recovery'][0] is not None) and (info['SCR_Onsets'][1] is not None) and (not pd.isna(info['SCR_Recovery'][0])) and (not pd.isna(info['SCR_Onsets'][1])) :
            peak = int(info['SCR_Recovery'][0])
            peak_ = int(info['SCR_Onsets'][1])
            if (peak_- peak) > (SAMPLING_RATE_EDA*TRIAL_LEN):
                break

    eda_low = eda_test[peak:peak_]
    eda = eda_low[-TRIAL_LEN*SAMPLING_RATE_EDA:]

    return ecg,eda

In [5]:
ecg_high, eda_high = generate_fake_eda_hr_shock()
ecg_low,eda_low = generate_fake_eda_hr_noshock()

In [6]:
df = pd.read_csv('csv_shock.csv')
df.drop(columns=df.columns[0], axis=1, inplace=True)
df_subj = df['subject'].unique()

In [7]:
df[df['subject']==2]

,subject,trial,rating,shock
0,2,1,3.0,False
1,2,2,3.0,False
2,2,3,3.0,False
3,2,4,3.0,False
4,2,5,3.0,False
...,...,...,...,...
155,2,156,1.0,False
156,2,157,1.0,False
157,2,158,1.0,False
158,2,159,1.0,False


In [11]:
for subject in df_subj:

    df_eda = list()
    df_hr = list()
    #df_eda = [list(range(SAMPLING_RATE_EDA))]
    df_tmp = df[df['subject']==subject]

    for index,row in df_tmp.iterrows():
        print('trial '+str(index))
        #multiply each trial *10 --> 1600 trials
        for j in range(0,10):
            j=j+1
            if row['rating'] > THRESHOLD_BINARY:
                hr,eda = generate_fake_eda_hr_shock()
            else:
                hr,eda = generate_fake_eda_hr_noshock()
            df_eda.append(eda)
            df_hr.append(hr)

    df_hr= pd.DataFrame(df_hr)
    df_hr.to_csv(r'data_fake/hr_csv/'+str(subject)+'_hr.csv', index=False)
    df_eda = pd.DataFrame(df_eda)
    df_eda.to_csv(r'data_fake/eda_csv/'+str(subject)+'_eda.csv', index=False)
    print('subject'+str(subject)+' done')

trial 0
trial 1
trial 2
trial 3
trial 4
trial 5
trial 6
trial 7
trial 8
trial 9
trial 10
trial 11
trial 12
trial 13
trial 14
trial 15
trial 16
trial 17
trial 18
trial 19
trial 20
trial 21
trial 22
trial 23
trial 24
trial 25
trial 26
trial 27
trial 28
trial 29
trial 30
trial 31
trial 32
trial 33
trial 34
trial 35
trial 36
trial 37
trial 38
trial 39
trial 40
trial 41
trial 42
trial 43
trial 44
trial 45
trial 46
trial 47
trial 48
trial 49
trial 50
trial 51
trial 52
trial 53
trial 54
trial 55
trial 56
trial 57
trial 58
trial 59
trial 60
trial 61
trial 62
trial 63
trial 64
trial 65
trial 66
trial 67
trial 68
trial 69
trial 70
trial 71
trial 72
trial 73
trial 74
trial 75
trial 76
trial 77
trial 78
trial 79
trial 80
trial 81
trial 82
trial 83
trial 84
trial 85
trial 86
trial 87
trial 88
trial 89
trial 90
trial 91
trial 92
trial 93
trial 94
trial 95
trial 96
trial 97
trial 98
trial 99
trial 100
trial 101
trial 102
trial 103
trial 104
trial 105
trial 106
trial 107
trial 108
trial 109
trial 110


In [12]:
df_eda

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,1.060292,1.058419,1.057294,1.056880,1.057063,1.057731,1.058770,1.060068,1.061510,1.062985,...,1.032834,1.031511,1.029761,1.027551,1.024967,1.022239,1.019605,1.017303,1.015572,1.014649
1,1.057777,1.058292,1.059439,1.061104,1.063175,1.065538,1.068081,1.070691,1.073254,1.075657,...,1.052904,1.050654,1.048778,1.047341,1.046352,1.045812,1.045723,1.046087,1.046907,1.048184
2,1.049161,1.047947,1.046797,1.045726,1.044747,1.043873,1.043119,1.042497,1.042021,1.041706,...,1.001598,0.999308,0.996056,0.991955,0.987293,0.982362,0.977454,0.972863,0.968881,0.965800
3,1.040600,1.040687,1.040650,1.040472,1.040131,1.039610,1.038889,1.037956,1.036861,1.035679,...,1.071287,1.068984,1.066934,1.065177,1.063751,1.062694,1.062047,1.061847,1.062133,1.062892
4,1.021609,1.022888,1.024037,1.025034,1.025860,1.026493,1.026914,1.027100,1.027033,1.026690,...,1.483615,1.482089,1.480279,1.478240,1.476061,1.473827,1.471627,1.469548,1.467676,1.466098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,1.130474,1.127460,1.124415,1.121397,1.118461,1.115665,1.113067,1.110722,1.108688,1.107022,...,1.059376,1.056691,1.053717,1.050599,1.047483,1.044513,1.041837,1.039598,1.037942,1.037016
1596,1.089845,1.090237,1.090334,1.090195,1.089877,1.089437,1.088934,1.088424,1.087965,1.087615,...,1.061837,1.060982,1.059992,1.058894,1.057727,1.056529,1.055339,1.054195,1.053137,1.052202
1597,0.960348,0.957886,0.956581,0.956664,0.958164,0.960812,0.964317,0.968391,0.972742,0.977080,...,1.462437,1.464909,1.467160,1.469025,1.470339,1.470980,1.470989,1.470445,1.469429,1.468020
1598,1.059709,1.061963,1.063885,1.065355,1.066312,1.066811,1.066915,1.066691,1.066205,1.065520,...,1.559855,1.557694,1.554885,1.551354,1.547026,1.541873,1.536090,1.529945,1.523701,1.517623
